In [2]:
import tweepy
from tweepy import OAuthHandler

ImportError: No module named tweepy

In [11]:
import tweepy
from tweepy import OAuthHandler

consumer_key = "cNbkKHve3645xLIVgJjQbrWg6"
consumer_secret = "B1SWKCIMFQjhh3Y5YGstKMvl8t7IAPTha1jxXy5b3IC6K8wz7w"
access_token = "3720533727-cBbDc2bDnDSU6f9SuC3ti4oQcfTkMk7VbVmh9ZW"
access_secret = "k2aioMNx3tCfw6jW9teksQ98WllWbKogVQF4YW0sRYTuW"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [12]:
user = api.me()
 
print('Name: ' + user.name)
print('Location: ' + user.location)
print('Friends: ' + str(user.followers_count))
print('Created: ' + str(user.created_at))
print('Description: ' + str(user.description))

Name: twiterbot_noticies
Location: 
Friends: 1
Created: 2015-09-20 19:16:41
Description: 


In [20]:
for status in tweepy.Cursor(api.home_timeline).items(1):
    print(status.text) 

Aprenent de manera divertida a l'assignatura de Cloud Computing! :) #perfectlife #loveupc #lovedacloud


In [21]:
import json

for status in tweepy.Cursor(api.home_timeline).items(1):
    print(json.dumps(status._json, indent=2))

{
  "contributors": null, 
  "truncated": false, 
  "text": "Aprenent de manera divertida a l'assignatura de Cloud Computing! :) #perfectlife #loveupc #lovedacloud", 
  "is_quote_status": false, 
  "in_reply_to_status_id": null, 
  "id": 857957627208359936, 
  "favorite_count": 0, 
  "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", 
  "retweeted": false, 
  "coordinates": null, 
  "entities": {
    "symbols": [], 
    "user_mentions": [], 
    "hashtags": [
      {
        "indices": [
          68, 
          80
        ], 
        "text": "perfectlife"
      }, 
      {
        "indices": [
          81, 
          89
        ], 
        "text": "loveupc"
      }, 
      {
        "indices": [
          90, 
          102
        ], 
        "text": "lovedacloud"
      }
    ], 
    "urls": []
  }, 
  "in_reply_to_screen_name": null, 
  "in_reply_to_user_id": null, 
  "retweet_count": 0, 
  "id_str": "857957627208359936", 
  "favorited": false, 
  "

In [22]:
for friend in tweepy.Cursor(api.friends).items(1):
    print(json.dumps(friend._json, indent=2))

{
  "follow_request_sent": false, 
  "has_extended_profile": false, 
  "profile_use_background_image": true, 
  "live_following": false, 
  "default_profile_image": false, 
  "id": 95695959, 
  "profile_background_image_url_https": "https://pbs.twimg.com/profile_background_images/653258374046478336/dw7DPLlC.jpg", 
  "translator_type": "regular", 
  "verified": true, 
  "blocked_by": false, 
  "profile_text_color": "333333", 
  "muting": false, 
  "profile_image_url_https": "https://pbs.twimg.com/profile_images/857140669839798273/QhREDVHI_normal.jpg", 
  "profile_sidebar_fill_color": "FFFFFF", 
  "entities": {
    "url": {
      "urls": [
        {
          "url": "https://t.co/FWP1q1SrbD", 
          "indices": [
            0, 
            23
          ], 
          "expanded_url": "http://www.barcelona.cat", 
          "display_url": "barcelona.cat"
        }
      ]
    }, 
    "description": {
      "urls": []
    }
  }, 
  "followers_count": 325976, 
  "profile_sidebar_border_col

In [17]:
from nltk.tokenize import word_tokenize

tweet = 'RT @JordiTorresBCN: just an example! :D http://JordiTorres.Barcelona #masterMEI'

print(word_tokenize(tweet))

['RT', '@', 'JordiTorresBCN', ':', 'just', 'an', 'example', '!', ':', 'D', 'http', ':', '//JordiTorres.Barcelona', '#', 'masterMEI']


In [18]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @JordiTorresBCN: just an example! :D http://JordiTorres.Barcelona #masterMEI'
print(preprocess(tweet))

['RT', '@JordiTorresBCN', ':', 'just', 'an', 'example', '!', ':D', 'http://JordiTorres.Barcelona', '#masterMEI']


In [19]:
user

User(follow_request_sent=False, has_extended_profile=False, profile_use_background_image=True, _json={u'follow_request_sent': False, u'has_extended_profile': False, u'profile_use_background_image': True, u'profile_text_color': u'333333', u'default_profile_image': True, u'suspended': False, u'id': 3720533727, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': False, u'translator_type': u'none', u'profile_location': None, u'profile_image_url_https': u'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', u'profile_sidebar_fill_color': u'DDEEF6', u'entities': {u'description': {u'urls': []}}, u'followers_count': 1, u'profile_sidebar_border_color': u'C0DEED', u'id_str': u'3720533727', u'profile_background_color': u'C0DEED', u'needs_phone_verification': False, u'listed_count': 0, u'status': {u'contributors': None, u'truncated': False, u'text': u'comen\xe7ant el tfg...', u'is_quote_status': False, u'in_reply_to_